<a href="https://colab.research.google.com/github/NormLorenz/ai-llm-youtube-transcription-utility/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convert the YouTube Transaction Utility to Colab
## Run in Google Colab in anticipation of using AI to:
* correct noun casing
* correct sentence ending - either a period, a question mark or a exclamation mark
* correct word spelling

In [ ]:
# installs

!pip install openai python-dotenv google-generativeai anthropic youtube_transcript_api

In [ ]:
# imports

import os
import io
import sys
import json
import requests
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess
from google.colab import userdata
from typing import Dict, List, Union
from youtube_transcript_api import YouTubeTranscriptApi
import re

In [ ]:
# get the utilities module from github

%%bash
wget https://raw.githubusercontent.com/NormLorenz/ai-llm-youtube-transcription-utility/refs/heads/main/utilities.py
wget https://raw.githubusercontent.com/NormLorenz/ai-llm-youtube-transcription-utility/refs/heads/main/README.md

In [ ]:
# add /content to the Python path to allow importing local modules.
# this ensures that Python can find 'utilities.py' as a module.
sys.path.append('/content')

# list content of /content for verification (optional).
display(os.listdir('/content'))

# import utilities.py
import utilities

In [ ]:
# keys

openai_api_key = userdata.get("OPENAI_API_KEY")
claude_api_key = userdata.get("ANTHROPIC_API_KEY")
google_api_key = userdata.get("GOOGLE_API_KEY")
hugging_face_token = userdata.get("HF_TOKEN")

In [ ]:
# initialize

openai = OpenAI(api_key=openai_api_key)
claude = anthropic.Anthropic(api_key=claude_api_key)
google.generativeai.configure(api_key=google_api_key)

OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-5-haiku-latest"
GOOGLE_MODEL = "gemini-2.5-flash-lite"

In [ ]:
# create a system message

def system_message():
    system_message = "You are a German language tutor. Your task is to review and correct a Python list of dictionaries that contain "
    system_message += "a time stamp and a German sentence. The sentence may have incorrect casing for German nouns and may have incorrect "
    system_message += "or missing period, question mark or explanation mark. Also please correct any casing at the start of the sentence and "
    system_message += "and any misspelled words."
    return system_message


In [ ]:
# create a user prompt

def user_prompt(items: List[Dict[str, str]]):
    user_prompt = "Please analyze the following and only return a Python list of dictionaries: \n"
    user_prompt += str(items)
    return user_prompt

In [ ]:
# stream an openai response

def stream_gpt(items: List[Dict[str, str]]):
    messages =  [
        {"role": "system", "content": system_message()},
        {"role": "user", "content": user_prompt(items)}
    ]
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages, stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```python\n','').replace('```','')

In [ ]:
sample_data = [
    {"start": "00:01", "text": "ich habe ein buch gelesen"},
    {"start": "00:05", "text": "dass ist ein haus"},
    {"start": "00:10", "text": "wie geht es dir"}
]

print("Calling stream_gpt with sample data...")
for chunk in stream_gpt(sample_data):
    print(chunk)

In [ ]:
# stream a antrophic response

def stream_claude(items: List[Dict[str, str]]):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message(),
        messages=[{"role": "user", "content": user_prompt(items)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```python\n','').replace('```','')

In [ ]:
sample_data = [
    {"start": "00:01", "text": "ich habe ein buch gelesen"},
    {"start": "00:05", "text": "dass ist ein haus"},
    {"start": "00:10", "text": "wie geht es dir"}
]

print("Calling stream_claude with sample data...")
for chunk in stream_claude(sample_data):
    print(chunk)

In [ ]:
# stream a google response

def stream_google(items: List[Dict[str, str]]):
    reply = ""
    gemini = google.generativeai.GenerativeModel(
        model_name=GOOGLE_MODEL,
        system_instruction=system_message()
    )
    response = gemini.generate_content(
        user_prompt(items),
        stream=True
    )
    for chunk in response:
        if chunk.text:
            reply += chunk.text
            yield reply.replace('```python\n','').replace('```','')

In [ ]:
sample_data = [
    {"start": "00:01", "text": "ich habe ein buch gelesen"},
    {"start": "00:05", "text": "dass ist ein haus"},
    {"start": "00:10", "text": "wie geht es dir"}
]

print("Calling stream_google with sample data...")
for chunk in stream_google(sample_data):
    print(chunk)

In [ ]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    elif model=="Gemini":
        result = stream_google(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far

In [ ]:
def execute_python(code):
        try:
            output = io.StringIO()
            sys.stdout = output
            exec(code)
        finally:
            sys.stdout = sys.__stdout__
        return output.getvalue()

In [ ]:
from gradio.components import html
from gradio.utils import T
"""Gradio UI for the YouTube Transcript Utility."""

from typing import List, Dict, Union, Tuple
import gradio as gr

css: str = """
  #my_html_box { border: 2px solid blue; padding: 10px; }
"""

js: str  = """
function() {
    const table = document.querySelector('table');
    const range = document.createRange();
    range.selectNode(table);
    window.getSelection().removeAllRanges();
    window.getSelection().addRange(range);
    document.execCommand('copy');
    alert('Table copied to clipboard!');
    window.getSelection().removeAllRanges();
    return [];
    }
"""

sample_url: str = 'https://www.youtube.com/watch?v=L6HnBjnkKmM&list=PLCCi8icw2DAO1qZGH7heUyTB8bJP1ICZ5&index=28'

def clear_fields() -> Tuple[None, str, str]:
    return (None, "", "")

def mock_original_data(url: str) -> List[Dict[str, str]]:
    return [
      {"start": "00:01", "text": "ich habe ein buch gelesen"},
      {"start": "00:05", "text": "dass ist ein haus"},
      {"start": "00:10", "text": "wie geht es dir"}
    ]

def mock_modified_data(url: str) -> List[Dict[str, str]]:
    return [
      {"start": "00:01", "text": "Ich habe ein Buch gelesen."},
      {"start": "00:05", "text": "Das ist ein Haus."},
      {"start": "00:10", "text": "Wie geht es dir?"}
    ]

def build_table(items: List[Dict[str, str]]) -> str:
    html: str = "<table width=100%>\n"
    html += "  <tbody>\n"
    for item in items:
        html += f"    <tr><td>{item['start']}</td><td>{item['text']}</td></tr>\n"
    html += "  </tbody>\n"
    html += "</table>"
    return html

def build_text(items: List[Dict[str, str]]) -> str:
    return str(items)

def fetch_transcript(url: str, mock: bool, model: str) -> Tuple[str, str]:
    if not url:
        return "", "<p style='color: orange;'>The YouTube Video URL field can't be empty!</p>"
    else:
        transcript_original: List[Dict[str, str]] = mock_original_data(url) if mock else utilities.get_transcript(url)
        transcript_modified = mock_modified_data(url) # if mock else stuff here
        list_original: str = build_text(transcript_original)
        html_modified: str = build_table(transcript_modified)
        return list_original, html_modified

with gr.Blocks(css=css) as ui:

    gr.Markdown("## YouTube Transcript Utility")
    gr.Markdown("### Creates a HTML table that can be copied and pasted into a Windows OneNote application")

    with gr.Row():
        url = gr.Textbox(label="YouTube Video URL:", value=sample_url)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude", "Gemini"], label="AI Model Name:", value="GPT")
    with gr.Row():
        mock = gr.Checkbox(label="Use Mock Data", value=True)
    with gr.Row():
        fetch = gr.Button("Fetch", variant="primary")
        copy = gr.Button("Copy")
        clear = gr.Button("Clear")
    with gr.Row():
        text_original = gr.TextArea(label="Original Transcript")
        html_modified = gr.HTML(elem_id="my_html_box")

    fetch.click(fetch_transcript, inputs=[url, mock, model], outputs=[text_original, html_modified])
    copy.click(fn=None, inputs=[], outputs=[], js=js)
    clear.click(clear_fields, inputs=[], outputs=[url, text_original, html_modified])

ui.launch(inbrowser=True, debug=True)
